In [6]:
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tag import pos_tag 
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from textblob import TextBlob
from nltk.corpus import stopwords 
import re,string
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from bs4 import BeautifulSoup
# from spellchecker import SpellChecker
import emoji
import pandas as pd
import numpy as np
import nltk
import googletrans
from googletrans import Translator
from sklearn import datasets, linear_model
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from nltk.corpus import wordnet
from sklearn.metrics import classification_report

In [7]:
import pandas as pd

In [8]:
# Read the CSV file and display the first few rows
df_csv = pd.read_csv('Love_Island_The_Game_play_store.csv')
df_US = pd.read_csv('Love_Island_The_Game_app_store_US.csv')
df_US.rename(columns={"userName": "user_name"}, inplace=True)
df_US.rename(columns={"date": "review_date"}, inplace=True)
df_US.rename(columns={"developerResponse": "developer_response"}, inplace=True)
df_US.rename(columns={"review": "review_description"}, inplace=True)
Love_Island_The_Game_app_store_US = pd.concat([df_csv, df_US], ignore_index=True)

In [9]:
Love_Island_The_Game_app_store_US

,review_id,user_name,userImage,review_description,rating,thumbs_up,reviewCreatedVersion,review_date,developer_response,developer_response_date,appVersion,isEdited,title
0,3f2cf013-eecf-424a-acc2-9d4e9be8b17b,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Giving a 4 star only based off of how expensiv...,4,0.0,1.5.9,2025-03-18 16:52:19,NaN,NaN,1.5.9,NaN,NaN
1,52ad7021-8fd1-4e5c-9c05-8682ed98e978,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Love this game,5,0.0,1.5.9,2025-03-18 16:41:19,NaN,NaN,1.5.9,NaN,NaN
2,8fe328d2-6ac4-4664-b436-450b73ed156e,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,very interesting and keeps you locked in,5,0.0,1.5.9,2025-03-18 15:32:48,NaN,NaN,1.5.9,NaN,NaN
3,d84c263a-0786-4988-97e4-e0b54d61fff5,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Super like for this game. Why 4 stars? Cos I d...,4,0.0,1.5.9,2025-03-18 15:01:26,NaN,NaN,1.5.9,NaN,NaN
4,0d95b6b5-33c4-41f4-89ed-1d91f05a2a7b,A Google user,https://play-lh.googleusercontent.com/EGemoI2N...,Interesting but definitely fun,5,0.0,1.5.9,2025-03-18 10:57:06,NaN,NaN,1.5.9,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7960,NaN,Negrinchulintufly,NaN,Porque dice que esta en español y no tiene en ...,1,NaN,NaN,2022-06-02 07:00:11,NaN,NaN,NaN,False,Engaño
7961,NaN,hejs fidbdd,NaN,I’ve been an enjoyer of this game since season...,1,NaN,NaN,2022-07-23 23:46:41,NaN,NaN,NaN,False,The first game is better.
7962,NaN,😀😜🙄,NaN,As soon as I saw the set up of the game I knew...,1,NaN,NaN,2022-06-23 02:32:31,NaN,NaN,NaN,False,There’s no way this isn’t a prank
7963,NaN,Peanut Butter&J,NaN,"It’s a fun game, I’ve always been a huge fan o...",4,NaN,NaN,2022-03-14 20:11:15,NaN,NaN,NaN,False,Pretty fun!


In [10]:
Love_Island_The_Game_app_store_US.drop(columns=['userImage', 'user_name', 'reviewCreatedVersion','appVersion'], inplace=True)

In [11]:
Love_Island_The_Game_app_store_US.drop(columns=['isEdited'], inplace=True)

In [12]:
Love_Island_The_Game_app_store_US.drop(columns=['developer_response_date'], inplace=True)

# **Text LoweCase**

In [150]:
def text_lowercase(text):
    return text.lower()

Love_Island_The_Game_app_store_US['review_description'] = Love_Island_The_Game_app_store_US['review_description'].apply(text_lowercase)

# **Remove Punctuation**

In [153]:
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

Love_Island_The_Game_app_store_US['review_description'] = Love_Island_The_Game_app_store_US['review_description'].apply(remove_punctuation)

# **Text Classification**

## ROBERTa

In [4]:
%pip install tf-keras

   ---------------------------------------- 0.0/1.7 MB ? eta -:--:--
   ------ --------------------------------- 0.3/1.7 MB ? eta -:--:--
   ---------------------------------------- 1.7/1.7 MB 7.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [19]:
from transformers import pipeline

# Load language detection model
detector = pipeline("text-classification", model="papluca/xlm-roberta-base-language-detection")

# Function to classify a single text
def classify_text(text):
    if pd.isna(text):  # Handle NaN values
        return None
    result = detector(text, top_k=1, truncation=True)[0]  # Get top result
    return result['label']  # Extract predicted language

# Apply function to the 'review_description' column
Love_Island_The_Game_app_store_US['label'] = Love_Island_The_Game_app_store_US['review_description'].apply(classify_text)

# Display sample results
print(Love_Island_The_Game_app_store_US[['review_description', 'label']].head())



Device set to use cpu


                                  review_description label
0  giving a 4 star only based off of how expensiv...    en
1                                     love this game    en
2           very interesting and keeps you locked in    en
3  super like for this game why 4 stars cos i don...    en
4                     interesting but definitely fun    en


In [20]:
Love_Island_The_Game_app_store_US['label'].value_counts()

label
en    7663
sw     101
ur      76
hi      76
it       9
tr       7
es       6
pt       5
pl       4
nl       3
fr       3
ar       2
bg       2
de       2
zh       2
ja       1
el       1
ru       1
th       1
Name: count, dtype: int64

## RNN

In [22]:
# load json and create model
# Write the file name of the model
import numpy as np
import keras
import matplotlib.pyplot as plt
import re
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from string import ascii_lowercase
from keras.models import model_from_json
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing import sequence

In [23]:
from tensorflow.keras.models import model_from_json, Sequential

# Load the model architecture from JSON file
with open("lang_class_relu_model.json", "r") as json_file:
    loaded_model_json = json_file.read()

# Reconstruct the model and provide the custom objects mapping
model_classifier = model_from_json(loaded_model_json, custom_objects={'Sequential': Sequential})

# Load the trained weights into the model
model_classifier.load_weights("lang_class_relu_model_weights.h5")

# Compile the model before using it
model_classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

print("Model loaded successfully!")

Model loaded successfully!


c:\Users\jiyaa\OneDrive\Desktop\NLP\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [24]:
# Create character encoding dictionary
od = {ch: idx for idx, ch in enumerate(ascii_lowercase, 1)}

In [25]:
# Function to preprocess a word (convert to numerical sequence)
def preprocess_word(word):
    word = word.lower()  # Convert to lowercase
    word = re.sub(r"[^a-z]", "", word)  # Remove non-alphabetic characters
    seq = [od.get(char, 0) for char in word]  # Convert to numbers
    return seq

In [26]:
# Function to predict language for a single word
def predict_word_language(word, model):
    seq = preprocess_word(word)
    if not seq:  # If word is empty after cleaning, return "en" by default
        return "en"

    seq = [seq]  # Convert to list of lists for padding
    seq_padded = pad_sequences(seq, maxlen=10)  # Pad with a smaller maxlen
    seq_padded = np.array(seq_padded).reshape(1, 10, 1)  # Reshape for LSTM

    # Predict
    prediction = model.predict(seq_padded)

    return "hi" if prediction[0][0] > 0.5 else "en"

In [27]:
from collections import Counter

In [28]:
# Function to predict language for a full sentence
def predict_language(text, model):
    words = text.split()  # Split sentence into words
    predictions = [predict_word_language(word, model) for word in words]

    # Count occurrences of each label
    counter = Counter(predictions)
    return counter.most_common(1)[0][0]  # Return most frequent label

## Removing NaN values

In [36]:
# Drop rows where 'review_description' is NaN
Love_Island_The_Game_app_store_US = Love_Island_The_Game_app_store_US.dropna(subset=['review_description']) 

## Remove Empty Strings ('')

In [42]:
Love_Island_The_Game_app_store_US = Love_Island_The_Game_app_store_US[Love_Island_The_Game_app_store_US['review_description'].str.strip() != '']

In [43]:
df = Love_Island_The_Game_app_store_US[Love_Island_The_Game_app_store_US['label'] != 'en']

In [44]:
df.shape

(301, 8)

In [45]:
df['label'] = df['review_description'].apply(lambda x: predict_language(x, model_classifier))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━

C:\Users\jiyaa\AppData\Local\Temp\ipykernel_21472\1666984598.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['review_description'].apply(lambda x: predict_language(x, model_classifier))


In [46]:
df['label'].value_counts()

label
en    185
hi    116
Name: count, dtype: int64

In [ ]:
Love_Island_The_Game_app_store_US.update(df)

In [48]:
Love_Island_The_Game_app_store_US['label'].value_counts()

label
en    7848
hi     116
Name: count, dtype: int64

In [49]:
Love_Island_The_Game_app_store_US

,review_id,review_description,rating,thumbs_up,review_date,developer_response,title,label
0,3f2cf013-eecf-424a-acc2-9d4e9be8b17b,giving a 4 star only based off of how expensiv...,4,0.0,2025-03-18 16:52:19,NaN,NaN,en
1,52ad7021-8fd1-4e5c-9c05-8682ed98e978,love this game,5,0.0,2025-03-18 16:41:19,NaN,NaN,en
2,8fe328d2-6ac4-4664-b436-450b73ed156e,very interesting and keeps you locked in,5,0.0,2025-03-18 15:32:48,NaN,NaN,en
3,d84c263a-0786-4988-97e4-e0b54d61fff5,super like for this game why 4 stars cos i don...,4,0.0,2025-03-18 15:01:26,NaN,NaN,en
4,0d95b6b5-33c4-41f4-89ed-1d91f05a2a7b,interesting but definitely fun,5,0.0,2025-03-18 10:57:06,NaN,NaN,en
...,...,...,...,...,...,...,...,...
7960,NaN,porque dice que esta en español y no tiene en ...,1,NaN,2022-06-02 07:00:11,NaN,Engaño,en
7961,NaN,i’ve been an enjoyer of this game since season...,1,NaN,2022-07-23 23:46:41,NaN,The first game is better.,en
7962,NaN,as soon as i saw the set up of the game i knew...,1,NaN,2022-06-23 02:32:31,NaN,There’s no way this isn’t a prank,en
7963,NaN,it’s a fun game i’ve always been a huge fan of...,4,NaN,2022-03-14 20:11:15,NaN,Pretty fun!,en


# **Filter the dataframe greater than 2 words**

In [50]:
Love_Island_The_Game_app_store_US.shape

(7964, 8)

In [58]:
# Filter the dataframe to include only rows where the review description contains greater than 2 words.
Love_Island_The_Game_app_store_US = Love_Island_The_Game_app_store_US[
    Love_Island_The_Game_app_store_US['review_description'].apply(lambda x: len(x.split()) > 2)
]

In [57]:
Love_Island_The_Game_app_store_US.shape

(7071, 8)

## **Convert emojis into text**

In [59]:
import demoji
demoji.download_codes()

C:\Users\jiyaa\AppData\Local\Temp\ipykernel_21472\2319952594.py:2: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [120]:
def replace_emojis(text):
    emojis = demoji.findall(text)
    for emo, desc in emojis.items():
        text = text.replace(emo, f":{desc}:")
    return text

Love_Island_The_Game_app_store_US['review_description'] = Love_Island_The_Game_app_store_US['review_description'].apply(replace_emojis)


In [61]:
Love_Island_The_Game_app_store_US.to_csv('Love_Island_The_Game_app_store_US_final.csv', index=False)

# **Translation**

In [101]:
df = Love_Island_The_Game_app_store_US[Love_Island_The_Game_app_store_US['label'] != 'en']

In [102]:
df

,review_id,review_description,rating,thumbs_up,review_date,developer_response,title,label
146,608464c1-cf7c-41a2-87c9-8e51d171faae,chalu nahin hoti game,5,0.0,2025-02-13 07:36:58,NaN,NaN,hi
202,ddb71a9b-6f7f-443e-a505-87d50352182a,donne moi plus de diamond,5,0.0,2025-02-05 20:02:36,NaN,NaN,hi
440,ebb98e85-d2f0-4e55-a40f-3197d26117f9,isse kam ka option available tha nahi game ny...,1,0.0,2025-01-09 15:14:01,NaN,NaN,hi
880,2dbcaa83-555a-49fe-a04e-c8f56ee47192,i love jin,5,1.0,2024-10-10 01:21:05,NaN,NaN,hi
948,01c4b556-2f8e-4c3e-ade2-ff73ef8fbd40,yah bahut kutti game hai please please hum log...,1,0.0,2024-09-21 13:19:15,NaN,NaN,hi
964,b74db153-b92e-4dee-8aa4-079100437622,in this game i want only lewie and i dont want...,5,0.0,2024-09-15 20:37:02,NaN,NaN,hi
1008,d860dac0-5bce-4b4a-8d60-17ac9f183d6b,i love jin sm ahhhhhh,5,0.0,2024-09-05 05:28:14,NaN,NaN,hi
1015,eda8f63b-8f8a-40ea-bc88-179e7e3e995b,i miss talia,3,0.0,2024-09-03 10:38:20,NaN,NaN,hi
1078,14da713a-74e0-4b06-b966-6c09030993c8,brill gane lovin it,5,0.0,2024-08-19 18:07:32,NaN,NaN,hi
1093,2eba5a25-3db4-4a67-b548-eedd6c32b6b8,fun game cant deny,5,0.0,2024-08-15 09:02:54,NaN,NaN,hi


In [ ]:
import google.generativeai as genai
import pandas as pd
import time

# Configure the Generative AI API
genai.configure(api_key="API_KEY")

def extract_text(response):
    """Extracts and concatenates text parts from the response candidate."""
    candidate = response.candidates[0]
    # Assuming candidate.content.parts is a list of objects with a .text attribute
    return "".join([part.text for part in candidate.content.parts])

def translate_and_correct_grammar(text):
    model = genai.GenerativeModel('gemini-1.5-flash-001')
    
    # Step 1: Translate from Hindi to English
    translate_prompt = f"Translate the following Hindi text to English:\n\n{text}"
    translated_response = model.generate_content(translate_prompt)
    translated_text = extract_text(translated_response)
    
    # Step 2: Correct grammar of the translated text
    correct_prompt = f"Rewrite the following sentence and fix any grammar issues and if no grammer issues are there then just write the sentenece don't give any explaination:\n\n{translated_text}"
    corrected_response = model.generate_content(correct_prompt)
    corrected_text = extract_text(corrected_response)
    
    return corrected_text

def process_review(row):
    return translate_and_correct_grammar(row['review_description'])

# Assuming df is defined earlier in your notebook
# Initialize an empty DataFrame to store results
df_new = pd.DataFrame()

# Process reviews in batches of 7 to avoid rate limiting
for start_index in range(0, len(df), 7):
    time.sleep(50)  # Sleep for 50 seconds to avoid rate limiting
    df_next_7 = df.iloc[start_index:start_index+7].copy()
    
    # Apply the function to each row (gets directly the corrected text)
    df_next_7['review_description'] = df_next_7.apply(process_review, axis=1)
    df_new = pd.concat([df_new, df_next_7], ignore_index=True)

# Now, df_new['review_description'] will contain only the extracted text.

In [104]:
df_new

,review_id,review_description,rating,thumbs_up,review_date,developer_response,title,label
0,608464c1-cf7c-41a2-87c9-8e51d171faae,The game is not working. \n,5,0.0,2025-02-13 07:36:58,NaN,NaN,hi
1,ddb71a9b-6f7f-443e-a505-87d50352182a,"The text ""donne moi plus de diamond"" is French...",5,0.0,2025-02-05 20:02:36,NaN,NaN,hi
2,ebb98e85-d2f0-4e55-a40f-3197d26117f9,There was no better option available. I'm goin...,1,0.0,2025-01-09 15:14:01,NaN,NaN,hi
3,2dbcaa83-555a-49fe-a04e-c8f56ee47192,I love Jin. \n,5,1.0,2024-10-10 01:21:05,NaN,NaN,hi
4,01c4b556-2f8e-4c3e-ade2-ff73ef8fbd40,Please provide the sentence you would like me ...,1,0.0,2024-09-21 13:19:15,NaN,NaN,hi
5,b74db153-b92e-4dee-8aa4-079100437622,"In this game, I only want Lewie, and I don't w...",5,0.0,2024-09-15 20:37:02,NaN,NaN,hi
6,d860dac0-5bce-4b4a-8d60-17ac9f183d6b,I love Jin so much! \n,5,0.0,2024-09-05 05:28:14,NaN,NaN,hi
7,eda8f63b-8f8a-40ea-bc88-179e7e3e995b,I miss Talia. \n,3,0.0,2024-09-03 10:38:20,NaN,NaN,hi
8,14da713a-74e0-4b06-b966-6c09030993c8,"The Hindi text ""brill gane lovin it"" translate...",5,0.0,2024-08-19 18:07:32,NaN,NaN,hi
9,2eba5a25-3db4-4a67-b548-eedd6c32b6b8,"It's a fun game, I can't deny it. \n",5,0.0,2024-08-15 09:02:54,NaN,NaN,hi


In [105]:
Love_Island_The_Game_app_store_US.update(df_new)

In [107]:
Love_Island_The_Game_app_store_US.drop(columns=['label'], inplace=True)

In [108]:
Love_Island_The_Game_app_store_US.to_csv('Love_Island_The_Game_app_store_US_final_2.csv', index=False)

## **Spelling correction**

In [109]:
%pip install spello

Note: you may need to restart the kernel to use updated packages.


In [111]:
from spello.model import SpellCorrectionModel
sp = SpellCorrectionModel(language='en')
sp.load('en_large.pkl\en_large.pkl')

<>:3: SyntaxWarning: invalid escape sequence '\e'
<>:3: SyntaxWarning: invalid escape sequence '\e'
C:\Users\jiyaa\AppData\Local\Temp\ipykernel_21472\3118520573.py:3: SyntaxWarning: invalid escape sequence '\e'
  sp.load('en_large.pkl\en_large.pkl')
c:\Users\jiyaa\OneDrive\Desktop\NLP\.venv\Lib\site-packages\spello\model.py:301: UserWarning: This model was saved on spell<1.3.0. As such due to a bug in previous versions, none of customisations made to the config at the time of training were saved along with the model. It is recommended to load the model, apply all required customizations to config and save it again. E.g.

from spello.model import SpellCorrectionModel 
sp = SpellCorrectionModel(language='en')  
sp.load('/home/ubuntu/model.pkl')
sp.config.min_length_for_spellcorrection = 4 # default is 3
sp.config.max_length_for_spellcorrection = 12 # default is 15
sp.save(model_save_dir='/home/ubuntu/')

After this the model will load without any warnings

  warnings.warn("This model was

In [114]:
sp.spell_correct('i wnt to plei futbal')

{'original_text': 'i wnt to plei futbal',
 'spell_corrected_text': 'i went to play football',
 'correction_dict': {'wnt': 'went', 'plei': 'play', 'futbal': 'football'}}

In [118]:
Love_Island_The_Game_app_store_US['review_description'] = Love_Island_The_Game_app_store_US['review_description'].apply(lambda x: x.get('spell_corrected_text') if isinstance(x, dict) and 'spell_corrected_text' in x else x)

In [121]:
Love_Island_The_Game_app_store_US

,review_id,review_description,rating,thumbs_up,review_date,developer_response,title
0,608464c1-cf7c-41a2-87c9-8e51d171faae,The game is not working,5,0.0,2025-02-13 07:36:58,NaN,NaN
1,ddb71a9b-6f7f-443e-a505-87d50352182a,The text donne moi plus de diamond is French...,5,0.0,2025-02-05 20:02:36,NaN,NaN
2,ebb98e85-d2f0-4e55-a40f-3197d26117f9,There was no better option available I'm goin...,1,0.0,2025-01-09 15:14:01,NaN,NaN
3,2dbcaa83-555a-49fe-a04e-c8f56ee47192,I love Jin,5,1.0,2024-10-10 01:21:05,NaN,NaN
4,01c4b556-2f8e-4c3e-ade2-ff73ef8fbd40,Please provide the sentence you would like me ...,1,0.0,2024-09-21 13:19:15,NaN,NaN
...,...,...,...,...,...,...,...
7066,NaN,parque dice que esta en español y no than en e...,1,NaN,2022-06-02 07:00:11,NaN,Engaño
7067,NaN,i’ve been an enjoyed of this game since season...,1,NaN,2022-07-23 23:46:41,NaN,The first game is better.
7068,NaN,as soon as i saw the set up of the game i knew...,1,NaN,2022-06-23 02:32:31,NaN,There’s no way this isn’t a prank
7069,NaN,it’s a fun game i’ve always been a huge fan of...,4,NaN,2022-03-14 20:11:15,NaN,Pretty fun!


# **Lemmatization**

Lemmatization is commonly used in natural language processing (NLP) and information retrieval applications, where it can improve the accuracy and performance of text analysis and search algorithms. By converting words to their base form, lemmatization can reduce the dimensionality of the text data and allow the algorithms to focus on the most critical and relevant information in the text.

## using SpaCy

In [135]:
%pip install spacy 
!python -m spacy download en_core_web_sm

Note: you may need to restart the kernel to use updated packages.
     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     - -------------------------------------- 0.5/12.8 MB 5.6 MB/s eta 0:00:03
     ------------- -------------------------- 4.2/12.8 MB 14.8 MB/s eta 0:00:01
     ------------------- -------------------- 6.3/12.8 MB 16.1 MB/s eta 0:00:01
     --------------------------------- ----- 11.0/12.8 MB 15.6 MB/s eta 0:00:01
     --------------------------------------- 12.8/12.8 MB 16.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [136]:
import spacy 
  
# python -m spacy download en_core_web_sm 
nlp = spacy.load("en_core_web_sm")

In [142]:
# Process whole documents 
text = ("""My name is Shaurya Uppal. I enjoy writing 
      articles on GeeksforGeeks checkout my other 
      article by going to my profile section.""") 

# Create a Doc object 
doc = nlp(text) 

# Get the lemma for each token and join them to form the sentence
lemmatized_text = " ".join([token.lemma_ for token in doc])
print(lemmatized_text)


my name be Shaurya Uppal . I enjoy write 
       article on GeeksforGeeks checkout my other 
       article by go to my profile section .


In [143]:
Love_Island_The_Game_app_store_US['review_description'] = Love_Island_The_Game_app_store_US['review_description'].apply(
    lambda text: " ".join([token.lemma_ for token in nlp(text)])
)

In [144]:
Love_Island_The_Game_app_store_US

,review_id,review_description,rating,thumbs_up,review_date,developer_response,title
0,608464c1-cf7c-41a2-87c9-8e51d171faae,the game be not work,5,0.0,2025-02-13 07:36:58,NaN,NaN
1,ddb71a9b-6f7f-443e-a505-87d50352182a,the text donne moi plus de diamond be Fren...,5,0.0,2025-02-05 20:02:36,NaN,NaN
2,ebb98e85-d2f0-4e55-a40f-3197d26117f9,there be no well option available I be go th...,1,0.0,2025-01-09 15:14:01,NaN,NaN
3,2dbcaa83-555a-49fe-a04e-c8f56ee47192,I love Jin,5,1.0,2024-10-10 01:21:05,NaN,NaN
4,01c4b556-2f8e-4c3e-ade2-ff73ef8fbd40,please provide the sentence you would like I t...,1,0.0,2024-09-21 13:19:15,NaN,NaN
...,...,...,...,...,...,...,...
7066,NaN,parque dice que esta en español y no than en e...,1,NaN,2022-06-02 07:00:11,NaN,Engaño
7067,NaN,I ’ve be an enjoyed of this game since season ...,1,NaN,2022-07-23 23:46:41,NaN,The first game is better.
7068,NaN,as soon as I see the set up of the game I know...,1,NaN,2022-06-23 02:32:31,NaN,There’s no way this isn’t a prank
7069,NaN,it ’ a fun game I ’ve always be a huge fan of ...,4,NaN,2022-03-14 20:11:15,NaN,Pretty fun!


## using NLTK

In [146]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\jiyaa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [149]:
import nltk 
from nltk.stem import WordNetLemmatizer 

# Define a text string 
text = "This is a sample text. It contains some words that we can use for lemmatization." 

# Tokenize the text into individual words 
tokens = nltk.word_tokenize(text) 

# Create a WordNetLemmatizer object 
lemmatizer = WordNetLemmatizer() 

# Lemmatize each word and store the results in a list 
lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

# Join the lemmatized tokens to form a sentence 
lemmatized_sentence = " ".join(lemmatized_tokens)
print(lemmatized_sentence)

This is a sample text . It contains some word that we can use for lemmatization .


In [151]:
Love_Island_The_Game_app_store_US['review_description'] = Love_Island_The_Game_app_store_US['review_description'].apply(
    lambda text: " ".join([lemmatizer.lemmatize(word) for word in word_tokenize(text)])
)

In [154]:
Love_Island_The_Game_app_store_US

,review_id,review_description,rating,thumbs_up,review_date,developer_response,title
0,608464c1-cf7c-41a2-87c9-8e51d171faae,the game be not work,5,0.0,2025-02-13 07:36:58,NaN,NaN
1,ddb71a9b-6f7f-443e-a505-87d50352182a,the text donne moi plus de diamond be french n...,5,0.0,2025-02-05 20:02:36,NaN,NaN
2,ebb98e85-d2f0-4e55-a40f-3197d26117f9,there be no well option available i be go thro...,1,0.0,2025-01-09 15:14:01,NaN,NaN
3,2dbcaa83-555a-49fe-a04e-c8f56ee47192,i love jin,5,1.0,2024-10-10 01:21:05,NaN,NaN
4,01c4b556-2f8e-4c3e-ade2-ff73ef8fbd40,please provide the sentence you would like i t...,1,0.0,2024-09-21 13:19:15,NaN,NaN
...,...,...,...,...,...,...,...
7066,NaN,parque dice que esta en español y no than en e...,1,NaN,2022-06-02 07:00:11,NaN,Engaño
7067,NaN,i ’ ve be an enjoyed of this game since season...,1,NaN,2022-07-23 23:46:41,NaN,The first game is better.
7068,NaN,a soon a i see the set up of the game i know i...,1,NaN,2022-06-23 02:32:31,NaN,There’s no way this isn’t a prank
7069,NaN,it ’ a fun game i ’ ve always be a huge fan of...,4,NaN,2022-03-14 20:11:15,NaN,Pretty fun!


# **Removing Stopwords**

In [155]:
# importing stop words from nltk corpus
from nltk.corpus import stopwords
# printing the list of all possible stop words in english language
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [156]:
sample_text = """Natural language processing refers to the branch of computer science and more specifically,
                 the branch of artificial intelligence or AI concerned with giving computers the ability to understand 
                 text and spoken words in much the same way human beings can understand."""

# tokenize sample_text into words
sample_words = sample_text.split()

# identify all the stop words in sample text
found_stopwords = [word for word in sample_words if word in stopwords.words('english')]
print(found_stopwords)

['to', 'the', 'of', 'and', 'more', 'the', 'of', 'or', 'with', 'the', 'to', 'and', 'in', 'the', 'same', 'can']


In [157]:
# removing stop words
no_stopwords = [word for word in sample_words if word not in stopwords.words('english')]

# joining back splitted words
sample_text = " ".join(no_stopwords)
print(sample_text)

Natural language processing refers branch computer science specifically, branch artificial intelligence AI concerned giving computers ability understand text spoken words much way human beings understand.


In [161]:
stop_words = set(stopwords.words('english'))

def remove_stopwords(text):
    tokens = text.split()
    filtered_tokens = [word for word in tokens if word.lower() not in stop_words]
    return " ".join(filtered_tokens)

Love_Island_The_Game_app_store_US['review_description'] = Love_Island_The_Game_app_store_US['review_description'].apply(remove_stopwords)

In [162]:
Love_Island_The_Game_app_store_US

,review_id,review_description,rating,thumbs_up,review_date,developer_response,title
0,608464c1-cf7c-41a2-87c9-8e51d171faae,game work,5,0.0,2025-02-13 07:36:58,NaN,NaN
1,ddb71a9b-6f7f-443e-a505-87d50352182a,text donne moi plus de diamond french hindi tr...,5,0.0,2025-02-05 20:02:36,NaN,NaN
2,ebb98e85-d2f0-4e55-a40f-3197d26117f9,well option available go game story hope least...,1,0.0,2025-01-09 15:14:01,NaN,NaN
3,2dbcaa83-555a-49fe-a04e-c8f56ee47192,love jin,5,1.0,2024-10-10 01:21:05,NaN,NaN
4,01c4b556-2f8e-4c3e-ade2-ff73ef8fbd40,please provide sentence would like review need...,1,0.0,2024-09-21 13:19:15,NaN,NaN
...,...,...,...,...,...,...,...
7066,NaN,parque dice que esta en español en español si ...,1,NaN,2022-06-02 07:00:11,NaN,Engaño
7067,NaN,’ enjoyed game since season 2 seem like every ...,1,NaN,2022-07-23 23:46:41,NaN,The first game is better.
7068,NaN,soon see set game know rough ride right gate k...,1,NaN,2022-06-23 02:32:31,NaN,There’s no way this isn’t a prank
7069,NaN,’ fun game ’ always huge fan love island game ...,4,NaN,2022-03-14 20:11:15,NaN,Pretty fun!


## Removing Numbers

In [163]:
#regex module
import re

#original string
string1 = "Hello!James12,India2020"

pattern = r'[0-9]'

# Match all digits in the string and replace them with an empty string
new_string = re.sub(pattern, '', string1)

print(new_string)

Hello!James,India


In [166]:
Love_Island_The_Game_app_store_US['review_description'] = Love_Island_The_Game_app_store_US['review_description'].apply(lambda x: re.sub(pattern, '', x))

In [167]:
Love_Island_The_Game_app_store_US

,review_id,review_description,rating,thumbs_up,review_date,developer_response,title
0,608464c1-cf7c-41a2-87c9-8e51d171faae,game work,5,0.0,2025-02-13 07:36:58,NaN,NaN
1,ddb71a9b-6f7f-443e-a505-87d50352182a,text donne moi plus de diamond french hindi tr...,5,0.0,2025-02-05 20:02:36,NaN,NaN
2,ebb98e85-d2f0-4e55-a40f-3197d26117f9,well option available go game story hope least...,1,0.0,2025-01-09 15:14:01,NaN,NaN
3,2dbcaa83-555a-49fe-a04e-c8f56ee47192,love jin,5,1.0,2024-10-10 01:21:05,NaN,NaN
4,01c4b556-2f8e-4c3e-ade2-ff73ef8fbd40,please provide sentence would like review need...,1,0.0,2024-09-21 13:19:15,NaN,NaN
...,...,...,...,...,...,...,...
7066,NaN,parque dice que esta en español en español si ...,1,NaN,2022-06-02 07:00:11,NaN,Engaño
7067,NaN,’ enjoyed game since season seem like every s...,1,NaN,2022-07-23 23:46:41,NaN,The first game is better.
7068,NaN,soon see set game know rough ride right gate k...,1,NaN,2022-06-23 02:32:31,NaN,There’s no way this isn’t a prank
7069,NaN,’ fun game ’ always huge fan love island game ...,4,NaN,2022-03-14 20:11:15,NaN,Pretty fun!


# **Filter dataframe again**

In [169]:
Love_Island_The_Game_app_store_US.shape

(7071, 7)

In [170]:
# Filter the dataframe to include only rows where the review description contains greater than 2 words.
Love_Island_The_Game_app_store_US = Love_Island_The_Game_app_store_US[
    Love_Island_The_Game_app_store_US['review_description'].apply(lambda x: len(x.split()) > 2)
]

In [171]:
Love_Island_The_Game_app_store_US.shape

(6522, 7)

In [172]:
Love_Island_The_Game_app_store_US.to_csv('Love_Island_The_Game_app_store_US_final_4.csv', index=False)